In [ ]:
""" this notebook helps to create database schema table for given dataframe """

In [ ]:
# testing if query works in database table

import os
import sys
sys.path.insert(1,os.path.join(os.path.realpath('__file__').split("lacrosse-prediction")[0],'lacrosse-prediction'))


import pandas as pd
from etl.db_and_schema_helper import dev_db_cnn,engine

cnn = dev_db_cnn
cursor = cnn.cursor()
engine=engine

In [ ]:
from etl.data_helper import read_df_using_dtype_map

In [ ]:
query_column_types_map = {'INTEGER':'INT DEFAULT NULL','TEXT':'VARCHAR(90) DEFAULT NULL','REAL':'DOUBLE DEFAULT NULL'}

In [ ]:
CSV_FILE_PATH = '../data/final_stage/testing.csv'
TARGET_TABLE_NAME = 'testing'
unique_index_cols=[]

In [ ]:
# CSV_FILE_PATH = '../data/final_stage/college_ranking.csv'
# TARGET_TABLE_NAME = 'testing_College_Rankings'
# unique_index_cols=[]

In [ ]:
# CSV_FILE_PATH = '../data/final_stage/club_ranking.csv'
# TARGET_TABLE_NAME = 'testing_Club_Rankings'
# unique_index_cols=['club', 'rating', 'link', 'gender', 'class', 'season','webSource']

In [ ]:
# CSV_FILE_PATH = '../data/final_stage/camps_and_clinics.csv'
# TARGET_TABLE_NAME = 'testing_Camps_and_Clinics'
# unique_index_cols=['event','state', 'year', 'gender', 'added', 'division']


In [ ]:
input_df=read_df_using_dtype_map(CSV_FILE_PATH,{})
input_df.dtypes

In [ ]:
def replace_using_map(
        input_string:str,
        input_map:dict
        ):
    """ change column maps to query"""
    
    for key, val in input_map.items():
        input_string=input_string.replace(key,val)
    return input_string

In [ ]:
def append_to_query(query:str,input_string:str):
    part_0,part_1=query.split('\n)')
    return part_0+',\n'+f"{input_string}"+'\n)'

In [ ]:
def gen_validated_unique_index(input_df,index_cols=[],index_name='unique_index2'):
    """ validates and returns unique index """
    output_string = f' UNIQUE KEY `{index_name}` ( '
    for col in index_cols:
        if col not in input_df.columns:
            return None
        output_string=output_string +f' `{str(col)}`,'
    output_string=output_string[:-1]+') '
    return output_string


In [ ]:
def add_primary_key_to_query(
        input_query:str,
        key_col:str,
        key_col_type='INT',
        constraints='NOT NULL AUTO_INCREMENT',
        ):
    """append primary key part into query"""
    part1,part2=input_query.split('(\n')
    input_query=part1+'(\n'+f'"{key_col}" {key_col_type} {constraints},\n'+part2
    # part1= input_query.split('\n)')[0]
    # input_query = part1+',\n'+f'PRIMARY KEY ("{key_col}")'+'\n)'
    input_query=append_to_query(input_query,f'PRIMARY KEY ("{key_col}")')
    return input_query

In [ ]:
def gen_table_schema_create_query(
        input_df:object,
        table_name:str,
        column_types_map={},
        primary_key_col='id',
        primary_key_col_type="INT",
        primary_key_constraints='NOT NULL AUTO_INCREMENT',
        unique_index_columns=[],
        unique_index_name='UNIQUE_INDEX2'
    ):
    """ method to create table schema query """
    query = pd.io.sql.get_schema(input_df,table_name)
    query=replace_using_map(query,column_types_map)
    query=add_primary_key_to_query(
        query,
        primary_key_col,
        primary_key_col_type,
        primary_key_constraints
        )
    if len(unique_index_columns)> 0:
        unique_index_constraint = gen_validated_unique_index(input_df,unique_index_columns,unique_index_name)
        if unique_index_constraint is not None:
            query = append_to_query(query,unique_index_constraint)
    query = query.replace('"','`')
    return f"""{query}"""
    

In [ ]:
query=gen_table_schema_create_query(input_df,TARGET_TABLE_NAME,query_column_types_map,unique_index_columns=unique_index_cols)
# query=query.replace("`event` VARCHAR(90)","`event` VARCHAR(255)")
# query=query.replace("`link` VARCHAR(90)","`link` VARCHAR(255)")

print(query)

In [ ]:
# cursor.execute(query)

#### Workshop